# Ipyrest, GitLab Examples

These examples require an API token for GitLab, which is assumed to be stored as an environment variable named `GITLAB_TOKEN`. More about the GitLab API can be found here: https://docs.gitlab.com/ee/api/

In [ ]:
import os

In [ ]:
from ipyrest import Api

In [ ]:
TOKEN = os.environ['GITLAB_TOKEN']
server = 'https://gitlab.com/api/v4'

## Prerequisites

Check GitLab version and our snippets, and maybe create some if we we want to.

In [ ]:
! curl --header "PRIVATE-TOKEN: $TOKEN" https://gitlab.com/api/v4/version

In [ ]:
! curl --header "PRIVATE-TOKEN: $TOKEN" https://gitlab.com/api/v4/snippets

In [ ]:
! curl --request POST \
     --data '{"title": "This is yet another snippet", "content": "Hello world", "description": "Hello World snippet", "file_name": "test.txt", "visibility": "internal" }' \
     --header 'Content-Type: application/json' \
     --header "PRIVATE-TOKEN: $TOKEN" \
     https://gitlab.com/api/v4/snippets

## Version

In [ ]:
Api(f'{server}/version', headers={'PRIVATE-TOKEN': TOKEN})

## Snippets

https://docs.gitlab.com/ee/api/snippets.html

### Get Private Snippets (Slow & Cached)

In [ ]:
Api(f'{server}/snippets', timeout=20, headers={'PRIVATE-TOKEN': TOKEN})

In [ ]:
Api(f'{server}/snippets', timeout=20, headers={'PRIVATE-TOKEN': TOKEN}, cassette_path='snippets.yaml')

### Get Snippets with Simple Custom View

In [ ]:
from ipywidgets import Textarea, Layout
from ipyrest import Api
from ipyrest.responseviews import ResponseView

class IdsView(ResponseView):
    "ResponseView showing IDs from a list of JSON objects."
    name = 'IDs'
    mimetype_pats = ['application/json']
    def render(self, resp):
        self.data = [snippet['id'] for snippet in resp.json()]
        return Textarea(value=str(self.data), layout=Layout(width='100%', height='100px'))

api = Api(f'{server}/snippets',
          additional_views=[IdsView], timeout=20,
          headers={'PRIVATE-TOKEN': TOKEN},
          cassette_path='snippets.yaml')
api

### Get Snippets with Advanced Custom View

In [ ]:
from ipywidgets import HBox, VBox, Text, Button, Layout
from ipyrest import Api
from ipyrest.responseviews import ResponseView

class ActionableIdsView(ResponseView):
    "ResponseView showing IDs with some 'UI'."
    name = 'ActionableIDs'
    mimetype_pats = ['application/json']
    def render(self, resp):
        return VBox([
                    HBox([Text(str(snippet['id'])), 
                          Text(snippet['title']), 
                          Button(description='Delete (dummy)')]
                        ) 
            for snippet in resp.json()])

api = Api(f'{server}/snippets', headers={'PRIVATE-TOKEN': TOKEN},
          cassette_path='snippets.yaml',
          additional_views=[ActionableIdsView], timeout=20)
api

### Access Response Data

In [ ]:
from ipyrest import Api
api = Api(f'{server}/snippets', method='post', headers={'PRIVATE-TOKEN': TOKEN},
   data={"title": "This is a NEW snippet", 
         "content": "Hello world", 
         "description": "Hello World snippet", 
         "file_name": "test.txt",
         "visibility": "internal" })
api

In [ ]:
id = api.resp.json()['id']
id

In [ ]:
from ipyrest import Api
Api(f'{server}/snippets/{id}/user_agent_detail', headers={'PRIVATE-TOKEN': TOKEN})

### Get Public Snippets, Failing with Error 500 (?)

In [ ]:
Api(f'{server}/snippets/public', timeout=60, headers={'PRIVATE-TOKEN': TOKEN})